In [2]:
import numpy as np
import os
from codeStore import support_fun as spf


In [29]:
# case 1, ecoli (head Force) model in finite pipe. 
ellipse_centerx_list = [0.75, 0.5, 0]
rot_theta_list = np.linspace(0, 2, 11) # true theta=rot_theta_list*np.pi
rs1_list = (0.1, 0.2, 0.3)
PoiseuilleStrength_list = np.linspace(-0.5, 0.5, 11)
# PoiseuilleStrength_list = (0, )
job_dir = 'case1a'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
    
t_name = os.path.join(job_dir, 'run.sh')
n_pbs = 0
with open(t_name, 'w') as frun:
    # create .pbs file
    for ellipse_centerx_i in ellipse_centerx_list:
        for rot_theta_i in rot_theta_list:
            for rs1_i in rs1_list:
                if rs1_i + ellipse_centerx_i < 1:
                    job_name = 'headF_x%.2f_rot%.2f_rs1%.2f' % (ellipse_centerx_i, rot_theta_i, rs1_i)
                    t_name = os.path.join(job_dir, '%s.pbs' % job_name)
                    with open(t_name, 'w') as fpbs:
                        spf.write_pbs_head(fpbs, job_name)
                        for PoiseuilleStrength_i in PoiseuilleStrength_list:
                            output_name = '%s_P%.2f' % (job_name, PoiseuilleStrength_i)
                            fpbs.write('mpirun -n 24 python ')
                            fpbs.write(' ../head_Force_in_pipe.py ')
                            fpbs.write(' -f %s ' % output_name)
                            fpbs.write(' -sm %s ' % 'lg_rs')
                            fpbs.write(' -legendre_m %d ' % 3)
                            fpbs.write(' -legendre_k %d ' % 2)
                            fpbs.write(' -epsilon %f ' % 3)
                            fpbs.write(' -PoiseuilleStrength %f ' % PoiseuilleStrength_i)
                            fpbs.write(' -ecoli_tail_strength %f ' % 1)
                            fpbs.write(' -rs1 %f ' % rs1_i)
                            fpbs.write(' -rs2 %f ' % 0.1)
                            fpbs.write(' -ds %f ' % 0.01)
                            fpbs.write(' -ellipse_centerx %f ' % ellipse_centerx_i)
                            fpbs.write(' -ellipse_centery %f ' % 0)
                            fpbs.write(' -ellipse_centerz %f ' % 0)
                            fpbs.write(' -rot_theta %f ' % rot_theta_i)
                            fpbs.write(' -finite_pipe_length %f ' % 10)
                            fpbs.write(' -finite_pipe_cover %d ' % 1)
                            fpbs.write(' -finite_pipe_ntheta %d ' % 100)
                            fpbs.write(' -vtk_matname %s ' % 'pipe_20181209')
                            fpbs.write(' -ksp_max_it %d ' % 500)
                            fpbs.write(' > %s.txt \n\n' % output_name)
                    # write to .sh file
                    frun.write('qsub %s.pbs\n' % job_name)
                    n_pbs = n_pbs + 1
    
    frun.write('\n')
    print('n_pbs = ', n_pbs)

n_pbs =  88


In [61]:
# case 2, ecoli (head Force) model in finite pipe. motion loop
ellipse_centerx_list = (0.75, 0.5, 0)
rot_theta_list = (0, 1/3, ) # true theta=rot_theta_list*np.pi
rs1_list = (0.1, )
PoiseuilleStrength_list = (0, 0.05)
# PoiseuilleStrength_list = (0, )
PWD = os.getcwd()
job_dir = 'case2a'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
    
t_name = os.path.join(job_dir, 'run.sh')
n_pbs = 0
with open(t_name, 'w') as frun:
    # create .pbs file
    frun.write('t_dir=$PWD \n')
    for ellipse_centerx_i in ellipse_centerx_list:
        for rs1_i in rs1_list:
            if rs1_i + ellipse_centerx_i < 1:
                for rot_theta_i in rot_theta_list:
                    for PoiseuilleStrength_i in PoiseuilleStrength_list:
                        job_name = 'headF_x%.2f_rot%.2f_rs1%.2f_P%.2f' % \
                                   (ellipse_centerx_i, rot_theta_i, rs1_i, PoiseuilleStrength_i)
                        print(job_name)
                        t_path = os.path.join(job_dir, job_name)
                        if not os.path.exists(t_path):
                            os.makedirs(t_path)
                        t_name = os.path.join(t_path, '%s.pbs' % job_name)
                        with open(t_name, 'w') as fpbs:
                            spf.write_pbs_head(fpbs, job_name)                           
                            fpbs.write('mpirun -n 24 python ')
                            fpbs.write(' ../../motion_head_Force_pipe.py ')
                            fpbs.write(' -f %s ' % output_name)
                            fpbs.write(' -sm %s ' % 'lg_rs')
                            fpbs.write(' -legendre_m %d ' % 3)
                            fpbs.write(' -legendre_k %d ' % 2)
                            fpbs.write(' -epsilon %f ' % 3)
                            fpbs.write(' -PoiseuilleStrength %f ' % PoiseuilleStrength_i)
                            fpbs.write(' -ecoli_tail_strength %f ' % 1)
                            fpbs.write(' -rs1 %f ' % rs1_i)
                            fpbs.write(' -rs2 %f ' % 0.1)
                            fpbs.write(' -ds %f ' % 0.01)
                            fpbs.write(' -ellipse_centerx %f ' % ellipse_centerx_i)
                            fpbs.write(' -ellipse_centery %f ' % 0)
                            fpbs.write(' -ellipse_centerz %f ' % 0)
                            fpbs.write(' -rot_theta %f ' % rot_theta_i)
                            fpbs.write(' -finite_pipe_length %f ' % 10)
                            fpbs.write(' -finite_pipe_cover %d ' % 1)
                            fpbs.write(' -finite_pipe_ntheta %d ' % 100)
                            fpbs.write(' -vtk_matname %s ' % 'pipe_20181209')
                            fpbs.write(' -ksp_max_it %d ' % 500)
                            fpbs.write(' -max_iter %d ' % 500)
                            fpbs.write(' -eval_dt %f ' % 0.01)
                            fpbs.write(' > %s.txt \n\n' % output_name)
                        # write to .sh file
                        frun.write('cd $t_dir/%s\n' % job_name)
                        frun.write('qsub %s.pbs\n\n' % job_name)
                        n_pbs = n_pbs + 1
    
    frun.write('\n')
    print('n_pbs = ', n_pbs)

headF_x0.75_rot0.00_rs10.10_P0.00
headF_x0.75_rot0.00_rs10.10_P0.05
headF_x0.75_rot0.33_rs10.10_P0.00
headF_x0.75_rot0.33_rs10.10_P0.05
headF_x0.50_rot0.00_rs10.10_P0.00
headF_x0.50_rot0.00_rs10.10_P0.05
headF_x0.50_rot0.33_rs10.10_P0.00
headF_x0.50_rot0.33_rs10.10_P0.05
headF_x0.00_rot0.00_rs10.10_P0.00
headF_x0.00_rot0.00_rs10.10_P0.05
headF_x0.00_rot0.33_rs10.10_P0.00
headF_x0.00_rot0.33_rs10.10_P0.05
n_pbs =  12


In [55]:
('cd %s\n' % os.path.join(PWD, t_path))
# t_path

'cd /home/zhangji/stokes_flow_master/head_Force/case2a/headF_x0.00_rot0.33_rs10.10_P0.05\n'